In [ ]:
from google.colab import drive
import numpy as np
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install streamlit -q
!pip install numpy streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%writefile new_GUI.py
import streamlit as st

#Libraries to load and display dataset
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.utils import np_utils
from skimage.util import montage
from PIL import Image


from keras.models import Model
from tensorflow.keras.utils import img_to_array,array_to_img


#to display our images and run eagerly our nueral model
tf.config.run_functions_eagerly(True)

np.random.seed(0)

def main():
  page_bg_img ="""
    <style>
    [data-testid="stAppViewContainer"]{
    background-image: url("https://media.gettyimages.com/id/455445559/vector/graphic-of-dots-connected-by-lines-in-blue-and-white-tones.jpg?s=1024x1024&w=gi&k=20&c=TVQVB6K-jC3RA7lLMAdVxvnNOHKxPkYGIOFOdDgp0Ko=");  
    background-size: cover;
    }

    [data-testid="stHeader"]{
      background-color: rgba(0,0,0,0)
    }

    footer{visibility:visible;}
    footer:after{
      content: 'Developed by Group_24 ©';
      text-align: center;
      background-color: grey;
      display: block;
      position: center;
      color: black;
    }


    </style>   
    """
  st.markdown(page_bg_img, unsafe_allow_html=True)

  st.title("Medical Image Analysis and Classification")

  #To upload the image
  uploaded_file = st.file_uploader("**Choose an image to upload**")

  #To check if image is uploaded and further processing
  if uploaded_file is not None:
    img = Image.open(uploaded_file)
    st.image(img)

    #Creating the dropdown for choice between blood and breast models
    a = ["Please Select the category","Blood","Breast"]
    choice = st.selectbox('**Please Select the category in which uploaded image belongs.**', a)

    if choice == "Breast":
      breast_dict = {}
      breast_class_labels = [
                "malignant",
                "normal, benign"]

      #Loading our model
      inception_loaded_model = keras.models.load_model("/content/drive/MyDrive/BREAST_INCEPTION.h5")
      
      with st.expander("**Summary for Inception Model**", expanded=False):
        st.text_area(label='', value=inception_loaded_model.summary(print_fn=lambda x: st.text(x)),key="1")

      #Processing our uploaded image
      inc_test = np.asarray([img_to_array(array_to_img(img, scale=False).resize((75,75)))])
      results = inception_loaded_model.predict(inc_test)
      single_result = results[0]
      most_likely = int(np.argmax(single_result))
      inc_class_likely = single_result[most_likely]
      inc_class_label = breast_class_labels[most_likely]

      #Returning the Class label and likelyhood.
      st.write("**This image belongs to '{}' class and the Likelihood is: {:2f}**".format(inc_class_label,inc_class_likely))
      breast_dict.update({inc_class_label:inc_class_likely})

      #Loading our model
      resnet101_loaded_model = keras.models.load_model("/content/drive/MyDrive/BREAST_RESNET101.h5")

      with st.expander("**Summary for ResNet101 Model**", expanded=False):
        st.text_area(label='', value=resnet101_loaded_model.summary(print_fn=lambda x: st.text(x)),key="2")

      #Processing our uploaded image
      resnet101_test = np.asarray([img_to_array(array_to_img(img, scale=False).resize((32,32)))])
      results = resnet101_loaded_model.predict(resnet101_test)
      single_result = results[0]
      most_likely = int(np.argmax(single_result))
      rn101_class_likely = single_result[most_likely]
      rn101_class_label = breast_class_labels[most_likely]

      #Returning the Class label and likelyhood.
      st.write("**This image belongs to '{}' class and the Likelihood is: {:2f}**".format(rn101_class_label,rn101_class_likely))
      breast_dict.update({rn101_class_label:rn101_class_likely})

      #Selecting the best model and returning the class label
      breast_max_value = max(breast_dict.values())
      breast_max_key = list(filter(lambda x: breast_dict[x] == breast_max_value, breast_dict))[0]
      with st.expander("**Conclusion For Blood Dataset**", expanded=True):
        st.write("**After checking the uploaded image from the models, we can conclude that this image belongs to '{}' class and the Likelihood is: {:2f}**".format(breast_max_key,breast_max_value))

    elif choice == "Blood":
      blood_dict = {}
      blood_class_labels = [
                "basophil",
                "eosinophil",
                "erythroblast",
                "immature granulocytes(myelocytes, metamyelocytes and promyelocytes)",
                "lymphocyte",
                "monocyte",
                "neutrophil",
                "platelet"]

      #Loading our model
      cnn_loaded_model = keras.models.load_model("/content/drive/MyDrive/BLOOD_CNN.h5")

      with st.expander("**Summary for CNN Model**", expanded=False):
        st.text_area(label='', value=cnn_loaded_model.summary(print_fn=lambda x: st.text(x)),key="3")

      #Processing our uploaded image
      cnn_test = np.asarray([img_to_array(array_to_img(img, scale=False).resize((28,28)))])
      results = cnn_loaded_model.predict(cnn_test)
      single_result = results[0]
      most_likely = int(np.argmax(single_result))
      cnn_class_likely = single_result[most_likely]
      cnn_class_label = blood_class_labels[most_likely]

      #Returning the Class label and likelyhood.
      st.write("**This image belongs to '{}' class and the Likelihood is: {:2f}**".format(cnn_class_label,cnn_class_likely))
      blood_dict.update({cnn_class_label:cnn_class_likely})
      
      #Loading our model
      resnet50_loaded_model = keras.models.load_model("/content/drive/MyDrive/BLOOD_RESNET50.h5")

      with st.expander("**Summary for resnet50 Model**", expanded=False):
        st.text_area(label='', value=resnet50_loaded_model.summary(print_fn=lambda x: st.text(x)),key="4")

      #Processing our uploaded image
      resnet50_test = np.asarray([img_to_array(array_to_img(img, scale=False).resize((32,32)))])
      results = resnet50_loaded_model.predict(resnet50_test)
      single_result = results[0]
      most_likely = int(np.argmax(single_result))
      rn50_class_likely = single_result[most_likely]
      rn50_class_label = blood_class_labels[most_likely]

      #Returning the Class label and likelyhood.
      st.write("**This image belongs to '{}' class and the Likelihood is: {:2f}**".format(rn50_class_label,rn50_class_likely))
      blood_dict.update({rn50_class_label:rn50_class_likely})

      #Selecting the best model and returning the class label
      blood_max_value = max(blood_dict.values())
      blood_max_key = list(filter(lambda x: blood_dict[x] == blood_max_value, blood_dict))[0]
      with st.expander("**Conclusion For Blood Dataset**", expanded=True):
        st.write("**After checking the uploaded image from the models, we can conclude that this image belongs to '{}' class and the Likelihood is: {:2f}**".format(blood_max_key,blood_max_value))
    
    else:
      st.write("Please select between Blood and Breast.")
if __name__ == '__main__':
  main()

Overwriting new_GUI.py


In [ ]:
!streamlit run new_GUI.py & npx localtunnel --port 8501

npx: installed 22 in 11.899s
your url is: https://ninety-bears-change-34-125-29-216.loca.lt



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.29.216:8501

2023-03-15 10:20:13.852741: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 10:20:15.235571: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-15 10:20:15.235718: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dy